<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br><br><br>

# Listed Volatility and Variance Derivatives

**Wiley Finance (2017)**

Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [@dyjh](http://twitter.com/dyjh) | http://books.tpq.io

<img src="https://hilpisch.com/images/lvvd_cover.png" alt="Listed Volatility and Variance Derivatives" width="30%" align="left" border="0">

# Advanced Modeling of the VSTOXX Index

## Introduction

This chapter is a bit different in style compared to the other chapters about the VSTOXX volatility index and related derivatives. It introduces another, parsimonious model, which is called square-root jump diffusion (SRJD) to model the VSTOXX volatility index. The model, which is essentially an extension of the Gruenbichler and Longstaff (1996) model as analyzed in the previous chapter, is capable of reproducing prices of European options written on the VSTOXX reasonably well.

Two major enhancements characterize the SRJD model:

* **term structure**: it allows to capture the term structure as observed in the prices of futures on the VSTOXX index
* **jump component**: including a jump component allows to better replicate option prices in the short term

Adding these two components makes a market-consistent calibration of the model to a comprehensive set of European options on the VSTOXX index possible. For similar analyses and modeling approaches including jumps for the volatility index refer, for example, to Psychoyios (2005), Sepp (2008) or Psychoyios et al. (2010).

## Market Quotes for Call Options

Before we introduce the model, let us set the stage by revisiting market quotes for European call options written on the VSTOXX volatility index. First, we import the data.

In [ ]:
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)
path = './data/'
h5 = pd.HDFStore(path + 'vstoxx_data_31032014.h5', 'r')

We have both options data and futures data stored in this file with quotes from 31. March 2014.

In [ ]:
h5

Option market quotes are what we are concerned with for the moment.

In [ ]:
option_quotes = h5['options_data']
option_quotes['DATE'] = pd.to_datetime(option_quotes['DATE'])
option_quotes['MATURITY'] = pd.to_datetime(option_quotes['MATURITY'])
option_quotes.info()

In [ ]:
option_quotes.head()

At any given point in time, there are options on the VSTOXX available for eight maturities.

In [ ]:
mats = sorted(set(option_quotes['MATURITY']))
mats

The spot level of the VSTOXX index on 31. March 2014 has been 17.6639. 

In [ ]:
v0 = 17.6639

In what follows, we only want to plot call option market quotes which are not too far in-the-money nor out-of-the-money.

In [ ]:
tol = 0.4
to_plot = option_quotes[(option_quotes['STRIKE'] > (1 - tol) * v0)
                      & (option_quotes['STRIKE'] < (1 + tol) * v0)]

The following figure shows the VSTOXX European call option quotes which fulfill the requirements. The goal of this chapter is to replicate "all these option quotes" as good as possible.

In [ ]:
from pylab import mpl, plt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
markers = ['.', 'o', '^', 'v', 'x', 'D', 'd', '>', '<']
plt.figure(figsize=(10, 6));
for i, mat in enumerate(mats):
    strikes = to_plot[(to_plot['MATURITY'] == mat)]['STRIKE']
    prices = to_plot[(to_plot['MATURITY'] == mat)]['PRICE']
    plt.plot(strikes, prices, 'b%s' % markers[i], label=str(mat)[:10])
plt.legend();
plt.xlabel('strike');
plt.ylabel('option quote');

<p style="font-family: monospace;">VSTOXX European call option quotes on 31. March 2014.

## The SRJD Model

Given is a filtered probability space $\{\Omega,\mathcal{F},\mathbb{F},P\}$ representing uncertainty in the model economy with final date $T$ where $0<T<\infty$. $\Omega$ denotes the continuous state space, $\mathcal{F}$ a $\sigma-$algebra, $\mathbb{F}$ a filtration &mdash; i.e. a family of non-decreasing $\sigma-$algebras $\mathbb{F}\equiv\{\mathcal{F}_{t\in [0,T]}\}$ with $\mathcal{F}_{0}\equiv\{\emptyset,\Omega\}$ and $\mathcal{F}_{T}\equiv \mathcal{F}$ &mdash; and $P$ the real or objective probability measure.

In the SRJD model, which is an affine jump diffusion (see Duffie et al. (2000)), the risk-neutral dynamics of the VSTOXX volatility index are given by the following stochastic differential equation (SDE)


$$\begin{equation*}
dv_{t}=\kappa (\theta -v_{t})dt+\sigma \sqrt{v_{t}}dZ_{t} + J_{t} v_{t} dN_{t}  -r_{J}dt
\end{equation*}$$


The meaning of the variables and parameters is

* $v_{t}$ volatility index level at date $t$
* $\kappa$ speed of adjustment of $v_{t}$ to ...
* ... $\theta$, the long-term mean of the index
* $\sigma$ volatility coefficient of the index level
* $Z_{t}$ standard Brownian motion
* $J_{t}$ jump at date $t$ with distribution ...
* ... $\log(1+J_{t})\approx \mathbf{N}\left(\log(1+\mu)-\frac{\delta^{2}}{2},\delta^{2}\right)$
* $\mathbf{N}$ cumulative distribution function of a standard normal random variable
* $N_{t}$ Poisson process with intensity $\lambda$
* $r_{J}\equiv \lambda \cdot \left(e^{\mu + \delta^{2}/2}-1\right)$ drift correction for jump

The stochastic process for $v$ is adapted to the filtration $\mathbb{F}$. Moreover, $Z$ and $N$ are not correlated. The time $t$ value of a zero-coupon bond paying one unit of currency at $T,0\leq t < T,$ is $B_{t}(T) = e^{-r(T-t)}$ with $r \geq 0$ the constant risk-less short rate. 

By the Fundamental Theorem of Asset Pricing, the time $t$ value of an attainable, $\mathcal{F}_{T}-$measurable contingent claim $V_{T}\equiv h_{T}(X_{T})\geq 0$ (satisfying suitable integrability conditions) is given by arbitrage as

\begin{equation*}
V_{t}=\mathbf{E}^{Q}_{t}\left( B_{t}(T) V_{T} \right)
\end{equation*}

with $V_{0}=\mathbf{E}^{Q}_{0}\left( B_{0}(T) V_{T} \right)$ as the important special case for valuation purposes. $Q$ is a $P-$equivalent martingale measure. The contingent claim could be a European call option maturing at $T$ with payoff $V_T = h_{T}(v_{T})\equiv\max[v_{T}-K,0]$. It could also be a European put with payoff $V_T = h_{T}(v_{T})\equiv\max[K-v_{T},0]$. In both cases, $K$ is the fixed strike price of the option. 

To simulate the financial model, i.e. to generate numerical values for $v_{t}$, it has to be discretized. To this end, divide the given time interval $[0,T]$ in equidistant sub-intervals $\Delta t$ such that now $t \in \{0,\Delta t,2 \Delta t, ..., T\}$, i.e. there are $M+1$ points in time with $M\equiv T/\Delta t$. With $s=t-\Delta t$, a discretization of the continuous time market model is given by 

$$\begin{eqnarray*}
\tilde{v}_{t} &=& \tilde{v}_{s} + \kappa(\theta-\tilde{v}_{s}^{+}) \Delta t + \sigma\sqrt{\tilde{v}_{s}^{+}} \sqrt{\Delta t}z^1_{t} \nonumber \\
&+& \left(e^{\mu_{J}+\delta^{2}z^{2}_{t}}-1\right) \tilde{v}_{s}^+ y_{t} -r_J \Delta t \\
v_{t} &=& \tilde{v}_{t}^{+} 
\end{eqnarray*}$$

for $t \in \{\Delta t, ..., T\}$ with $x^+\equiv\max[x,0]$ and the $z^{n}_{t}$ being standard normally distributed and $y_t$ Poisson distributed. $z^{1}_{t}, z^{2}_{t}$ and $y_t$ are uncorrelated. This discretization scheme is an Euler discretization and is generally called *full trunction* scheme. See Lord et al. (2008) for an analysis of this and other biased discretization schemes for the square-root diffusion process.

## Term Structure Calibration

The first in the  calibration of the SRJD model, is with regard to the futures term structure.

### Futures Term Structure

It is difficult for parsimonious short rate models like Cox et al. (1985) to account for different term structures of the interest rate. A possible solution is the introduction of time-dependent parameters which, however, enlarges the number of parameters significantly, sacrificing at the same time the convenience of a limited number of economic parameters. Another solution is a *deterministic shift approach* according to Brigo and Mercurio (2001) which preserves the basic structure of the model with all its advantages and which nevertheless allows to better account for different term structures of the short rate.

In this section, we transfer the deterministic shift approach for short rate models of Brigo and Mercurio (2001) to the SRJD model. Since the square-root diffusion and the jumps are not correlated, we can first apply the approach to the diffusion part and use this enhanced component later on in conjunction with the jump component.

Before we present the theory, a look at the VSTOXX futures data first.

In [ ]:
futures_quotes = h5['futures_data']
futures_quotes['DATE'] = pd.to_datetime(futures_quotes['DATE'])
futures_quotes['MATURITY'] = pd.to_datetime(futures_quotes['MATURITY'])
futures_quotes.info()

The following figure presents the futures quotes for all eight maturities.

In [ ]:
ax = futures_quotes.plot(x='MATURITY', y='PRICE',
                         figsize=(10, 6), legend=False)
futures_quotes.plot(x='MATURITY', y='PRICE', style='ro', ax=ax);

<p style="font-family: monospace;">VSTOXX futures quotes on 31. March 2014.

Consider for now the square-root diffusion volatility model of Gruenbichler and Longstaff (1996) as presented in the previous chapter, which is formally the same as the short rate model of Cox et al. (1985) and which is a special case of the SRJD model

\begin{equation*}
dv_t = \kappa (\theta -v_{t})dt + \sigma \sqrt{v_{t}}dZ_{t}
\end{equation*}

We want to calibrate this model to the observed volatility term structure, given by the above presented set of prices for futures on the VSTOXX index with different maturities. We have to minimize for all considered times $t$ and a parameter set $\alpha = (\kappa,\theta,\sigma,v_{0})$ simultaneously the single differences

\begin{equation*}
\Delta f(0,t) \equiv f(0,t)-f^{GL96}(0,t;\alpha)
\end{equation*}

where $f(0,t)$ is the time $0$ market (instantaneous) forward volatility for time $t$ and the quantity $f^{GL96}(0,t;\alpha)$ is the model (instantaneous) forward volatility for time $t$ given parameter set $\alpha$.

Assume that there is a continuously differentiable volatility term structure function $F(0,t)$ available (i.e. there are infinitely many volatility futures prices). The forward volatility then is

\begin{equation*}
f(0,t)=\frac{\partial F(0,t)}{\partial t}
\end{equation*}

On the other hand, the model implied forward volatility is given as (see Brigo and Mercurio (2001))

\begin{eqnarray*}
f^{GL96}(0,t;\alpha) = \frac{\kappa \theta\left(e^{\gamma t}-1\right)}{2\gamma+(\kappa+\gamma)\left(e^{\gamma t}-1\right)} \nonumber \\
+v_{0} \frac{4 \gamma ^{2} e^{\gamma t}}{\left(2\gamma+(\kappa+\gamma)\left(e^{\gamma t}-1\right)\right)^{2}} 
\end{eqnarray*}

with

\begin{equation*}
\gamma \equiv \sqrt{\kappa^{2}+2\sigma^{2}}
\end{equation*}

The Python script ``srjd_fwd_calibration.py`` contains the Python code to calibrate the forward volatilities to the VSTOXX futures prices (see the appendix for the complete script). The beginning of the script is about library imports, importing the data sets and making some selections.

In [ ]:
!sed -n 8,28p scripts/srjd_fwd_calibration.py

The function ``srd_forwards()`` implements the forward formula from above for a given parameter set. 

In [ ]:
import sys
sys.path.append('scripts')

In [ ]:
import srjd_fwd_calibration as srjdf

In [ ]:
srjdf.srd_forwards??

To operationalize the calibration, we use the mean squared error (MSE) as our yardstick

\begin{equation*}
\min_{\alpha } \frac{1}{N} \sum_{n=1}^{N}\left( f_{n} - f_{n}^{GL96}(\alpha )\right)^{2}
\end{equation*}

which is to be minimized. Here, we assume that we have $N$ observations for the forward volatility. In Python this takes on the form of function ``srd_fwd_error()``.

In [ ]:
srjdf.srd_fwd_error??

Executing the script yields optimal parameters for the Gruenbichler and Longstaff (1996) model given the VSTOXX futures prices.

In [ ]:
%run scripts/srjd_fwd_calibration.py

In [ ]:
opt.round(3)

These optimal values can be used to calculate the model forward volatilities using function ``srd_forwards()``.

In [ ]:
from srjd_fwd_calibration import *
srd_fwds = srd_forwards(opt)
srd_fwds

The numerical differences to the market futures prices are:

In [ ]:
srd_fwds - forwards

The following figure compares the model futures prices (forward volatilities) with the VSTOXX futures market quotes. For longer maturities the fit is quite well.

In [ ]:
plt.figure(figsize=(10, 6));
plt.plot(ttms, forwards, 'b', label='market prices');
plt.plot(ttms, srd_fwds, 'ro', label='model prices');
plt.legend(loc=0);
plt.xlabel('time-to-maturity (year fractions)');
plt.ylabel('forward volatilities');

<p style="font-family: monospace;">VSTOXX futures market quotes vs. model prices.

Finally, we save the results from the term structure calibration for later use during the simulation of the model.

In [ ]:
import pickle
f = open('varphi', 'wb')  # open file on disk
## write ttms object and differences (varphi values) as dictionary
pickle.dump({'ttms': ttms, 'varphi': srd_fwds - forwards}, f)
f.close()  # close file

### Shifted Volatility Process

Assume that we are given a continuously differentiable futures price function (i.e. through splines interpolation of discrete futures prices for different maturities). We consider now the deterministically shifted volatility process (see Brigo and Mercurio (2001))

\begin{equation*}
\hat{v}_t \equiv v_t + \varphi(t,\alpha^*)
\end{equation*}

with $\varphi(t,\alpha^*) \equiv f(0,t)-f^{GL96}(0,t;\alpha^*)$, the difference at time $t$ between the market implied forward volatility and the model implied forward volatility after calibration, that is for the optimal parameter set $\alpha^*$. $\varphi(t,\alpha^*)$ corresponds to the differences (bars) in the following figure.

In [ ]:
plt.figure(figsize=(10, 6));
plt.bar(ttms, srd_fwds - forwards,
            width=0.05, label='$\\varphi(t,\\alpha^*)$')
plt.legend(loc=0)
plt.xlabel('time-to-maturity (year fractions)')
plt.ylabel('deterministic shift');

<p style="font-family: monospace;">Deterministic shift values to account for VSTOXX futures term structure.

The SRJD model discretization can now be adjusted as follows:

\begin{eqnarray}
\tilde{v}_{t} &=& \tilde{v}_{s} + \kappa(\theta-\tilde{v}_{s}^{+}) \Delta t + \sigma\sqrt{\tilde{v}_{s}^{+}} \sqrt{\Delta t}z^1_{t} \nonumber \\
&+& \left(e^{\mu_{J}+\delta^{2}z^{2}_{t}}-1\right)  \tilde{v}_{s}^+ y_{t} -r_J \Delta t  \label{eq:disc3} \\
\hat{v}_{t} &=& \tilde{v}_{t}^{+} + \varphi(t,\alpha^*) \label{eq:disc4}
\end{eqnarray}

This is consistent since the diffusion and jump parts are not correlated and since the jump part is added in a way that the first moment of the stochastic volatility process does not change.

## Option Valuation by Monte Carlo Simulation

This section implements Monte Carlo simulation procedures for the SRJD model.

### Monte Carlo Valuation

In what follows, the model option values are computed by Monte Carlo simulation (MCS). Given the discrete version of the financial model, the value of a European call option on the volatility index is estimated by MCS as follows:

<img src="https://hilpisch.com/lvvd_algo.png" align="center">

$V_0$ is the MCS estimator for the European call option value.

### Technical Implementation

In view of using a numerical method like MCS for all valuation and calibration tasks, the parametrization and implementation of the MCS algorithm plays an important role. Some major features of our implementation are:

* **discretization**: the algorithm uses the Euler discretization scheme which is an approximation only but which might bring performance benefits
* **random numbers**: for every single option valuation the seed can be held constant such that every option is valued with the same set of (pseudo-) random numbers
* **variance reduction**: both antithetic variates and moment matching (for the first two moments of the pseudo-random numbers) are used as generic variance reduction techniques
* **deterministic shift**: the deterministic shift values $\varphi$ are determined only once through a separate calibration and are held constant afterwards (even if model parameters change)

$\varphi$ only has to be deterministic and integrable on closed intervals (see Brigo and Mercurio (2001)), which is of course the case. For the approach to be valid, it is not important how we came up with the $\varphi$ originally.

The Python code for simulating the SRJD model is found in the script ``srjd_simulation.py`` (see the appendix for the complete script). The beginning of the script shows several imports, the definition of example parameters and also the cubic splines interpolation to be used for the estimation of the deterministic shift parameters.

In [ ]:
!sed -n 8,39p scripts/srjd_simulation.py

The Python function ``random_number_gen()`` generates arrays of standard normally distributed pseudo-random numbers using both antithetic variates and moment matching as generic variance reduction techniques.

In [ ]:
import srjd_simulation as srjds

In [ ]:
srjds.random_number_gen??

The major function of this script is ``srjd_simulation()`` which implements the Monte Carlo simulation for the SRJD model based on an Euler discretization scheme. The scheme used here is usually called *full truncation* scheme.

In [ ]:
srjds.srjd_simulation??

Finally, the function ``srjd_call_valuation()`` estimates the value of a European call option given the simulated volatility paths from ``srjd_simulation()``.

In [ ]:
srjds.srjd_call_valuation??

Executing the script yields a MCS estimator for the European call option with the parameters as assumed as in the script of about 1 currency unit.

In [ ]:
%run scripts/srjd_simulation.py

## Model Calibration

This section now calibrates the SRJD model to market quotes for European call options on VSTOXX futures. It considers calibrations to a single maturity as well as to multiple maturities.

### The Python Code

The calibration of the SRJD model is similar to the procedure for the Gruenbichler and Lonstaff (1996) square-root diffusion model as presented in the previous chapter. The major difference now is that we have to take into account more parameter values for the optimization. The Python code is contained in script ``srjd_model_calibration.py`` (see the appendix for the complete script). As usual a few imports and parameter definitions first.

In [ ]:
!sed -n 10,21p scripts/srjd_model_calibration.py

In what follows, we want to calibrate the model simultaneously to multiple maturities for the VSTOXX European call options. The valuation function ``srjd_valuation_function()`` therefore now calculates the differences between model and market values directly and returns an array with all differences (relative or absolute).

In [ ]:
import srjd_model_calibration as srjdc

In [ ]:
srjdc.srjd_valuation_function??

The error function ``srjd_error_function()`` for the SRJD model has to be enhanced compared to the SRD case to account for the additional parameters of the model.

In [ ]:
srjdc.srjd_error_function??

The same holds true for the calibration function ``srjd_model_calibration()`` itself. This function allows to select certain maturities for the the calibration.

In [ ]:
srjdc.srjd_model_calibration??

### Short Maturity

The addition of a jump component shall allow a better fit to short term call option market quotes. Therefore, consider the following calibration to the shortest option maturity available.

In [ ]:
from srjd_model_calibration import *
## read option data, allow for 30% moneyness tolerance
option_data = read_select_quotes(tol=0.3)
option_data

In [ ]:
%%time
opt_1 = srjd_model_calibration(option_data, p0=None,
                               rel=False, mats=['2014-4-18'])

The optimal parameter values are:

In [ ]:
opt_1

Using these optimal parameter values, add the model prices to the ``DataFrame`` object containing the option data.

In [ ]:
values = []
kappa, theta, sigma, lamb, mu, delta = opt_1
for i, option in option_data.iterrows():
    value = srjd_call_valuation(v0, kappa, theta, sigma,
                                lamb, mu, delta,
                                option['TTM'], r, option['STRIKE'],
                                M=M, I=I, fixed_seed=True)
    values.append(value)
option_data['MODEL'] = values

The following figure shows the calibration results graphically. Indeed, the fit seems to be quite good, reflecting a MSAE of about 0.0015 only.

In [ ]:
## selecting the data for the shortest maturity
os = option_data[option_data.MATURITY == '2014-4-18']
## selecting corresponding strike prices
strikes = os.STRIKE.values
## comparing the model prices with the market quotes
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(10, 6));
ax[0].plot(strikes, os.PRICE.values, label='market quotes');
ax[0].plot(strikes, os.MODEL.values, 'ro', label='model prices');
ax[0].legend()
ax[1].bar(strikes, os.MODEL.values - os.PRICE.values,
                        width=0.3);
ax[1].set_xlim(12.5, 23);

<p style="font-family: monospace;">Calibration of SRJD model to European call options on the VSTOXX for shortest option maturity (April 2014).

### Two Maturities

Let us proceed with the simultaneous calibration of the model to the May and July maturities. The MSE is also pretty low in this case (i.e. below 0.01).

In [ ]:
## read option data, allow for 17.5% moneyness tolerance
option_data = read_select_quotes(tol=0.175)
# option_data

In [ ]:
%%time
opt_2 = srjd_model_calibration(option_data, rel=False,
                            mats=['2014-5-16', '2014-7-18'])

The optimal parameter values are:

In [ ]:
opt_2

In what follows, we use the Python function ``plot_calibration_results()`` to generate the plots for the different valuation runs. This function allows for different numbers of sub-plots, i.e. when the number of option maturities is changed. It is mainly a generalization of the plotting code used above.

In [ ]:
srjdc.plot_calibration_results??

The following figure shows the results of the  calibration graphically. It is obvious that the SRJD model is able to account for multiple maturities at the same time which is mainly due to the term structure component introduced by the deterministic shift approach.

In [ ]:
plot_calibration_results(option_data, opt_2, ['2014-5-16', '2014-7-18'])

<p style="font-family: monospace;">Calibration of SRJD model to European call options on the VSTOXX for May and July 2014 maturities.

### Four Maturities

In a next step, we consider four maturities &mdash; before we try to calibrate the model to all eight maturities.

In [ ]:
mats = sorted(set(option_data['MATURITY']))
mats

The four maturities for this particular calibration run are:

In [ ]:
mats[::2]

For this calibration run, we use the optimal parameters from the previous calibration to two maturities. Obviously, the more options to calibrate the model to, the longer the procedure takes.

In [ ]:
%%time
opt_4 = srjd_model_calibration(option_data, p0=opt_2,
                               rel=False, mats=mats[::2])

In [ ]:
opt_4

Even calibrating the model to four maturities yields quite a good fit over these maturities as the following figure illustrates.

In [ ]:
plot_calibration_results(option_data, opt_4, mats[::2])

<p style="font-family: monospace;">Calibration of SRJD model to European call options on the VSTOXX for four maturities.

### All Maturities

Finally, let us attack the hardest calibration problem &mdash; the one involving all eight option maturities.

In [ ]:
%%time
opt_8_MSAE = srjd_model_calibration(option_data,
                                rel=False, mats=mats)

In [ ]:
opt_8_MSAE

The following shows that the fit is still reasonable for eight maturities and that many options.

In [ ]:
plot_calibration_results(option_data, opt_8_MSAE, mats)

<p style="font-family: monospace;">Calibration of SRJD model to European call options on the VSTOXX for all eight maturities (MSAE used).

To check whether there is a (larger) difference when we calibrate the model using relative differences (i.e. the MSRE) as yardstick, consider the following calibration run.

In [ ]:
%%time
opt_8_MSRE = srjd_model_calibration(option_data, p0=opt_8_MSAE,
                                    rel=True, mats=mats)

The following figure presents the results. They are not too dissimular to the ones obtained using the MSAE as yardstick. The major difference is the weighting of the options in that now those options with lower market quotes (higher strikes) get more weight.

In [ ]:
plot_calibration_results(option_data, opt_8_MSRE, mats)

<p style="font-family: monospace;">Calibration of SRJD model to European call options on the VSTOXX for all eight maturities (MSRE used).

## Conclusions

This chapter introduces a more sophisticated model, the so-called square-root jump diffusion model (SRJD), for the evolution of the VSTOXX volatility index over time. It enhances the Gruenbichler and Longstaff (1996) square-root diffusion model by two components: a log normally distributed *jump component* and a *determistic shift component*. While the first allows to better calibrate the model to short term option quotes, the latter makes it possible to take the volatility term structure &mdash; as embodied by the eight futures on the VSTOXX index &mdash; into account. All in all, the model yields good calibration results even in cases where all eight option maturities are accounted for.

## Python Scripts

### srjd_fwd_calibration.py

In [ ]:
!cat scripts/srjd_fwd_calibration.py

### srjd_simulation.py 

In [ ]:
!cat scripts/srjd_simulation.py

### srjd_model_calibration.py

In [ ]:
!cat scripts/srjd_model_calibration.py

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>